In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Import model to "**Console**"

This notebook explains the workflow for importing model to "**Console**". <br>
<br>
Instructions are described in [README.md](./README.md).

## Imports

In [ ]:
import copy
import errno
import json
from pathlib import Path

import jsonschema
from console_access_library.client import Client

## Load Configurations

Load the configuration file and set the variables.

In [ ]:
def validate_symlink(path: Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )


configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration
jsonschema.validate(json_load, json_schema)

# Set default network_type
if json_load.get("network_type") is None:
    json_load.update({"network_type": "0"})

## Load an instance of "**Console Access Library**" Client

Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [ ]:
%store -r
client_obj: Client = client_obj

## Import AI model

In [ ]:
# Get an instance of AI model API
ai_model_obj = client_obj.get_ai_model()

param_dict = copy.deepcopy(json_load)

# Call an API to import AI model into Console for AITRIOS
try:
    response = ai_model_obj.import_base_model(**param_dict)
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
print("Start to import the AI model." + " \n\tmodel_id: " + json_load["model_id"])

## Get AI model status

In [ ]:
def get_base_model_status(model_id: str):
    # AI model status on Console
    status_dictionary = {
        "00": "Conversion project created",
        "01": "Importing completed (Before conversion)",
        "02": "Converting...",
        "03": "Conversion failed",
        "04": "Converted",
        "05": "Adding to configuration",
        "06": "Conversion failed",
        "07": "Conversion completed",
        "11": "Saving",
    }
    # Flag for check
    exist_flag = False
    # Call an API for get AI model info
    try:
        response = ai_model_obj.get_base_model_status(model_id)
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    # Create output list
    if "projects" in response:
        project = response["projects"][0]
        if "versions" in project:
            version_status = project["versions"][0]["version_status"]
            exist_flag = True

    if exist_flag:
        return status_dictionary.get(
            version_status, "Unknown status '" + version_status + "'"
        )
    else:
        raise Exception("AI model is not found. (model_id: " + model_id + ")")


get_status = get_base_model_status(json_load["model_id"])
print(get_status + " \n\tmodel_id: " + json_load["model_id"])

## Convert AI model

In [ ]:
# Call an API to convert AI model on Console for AITRIOS
try:
    response = ai_model_obj.publish_model(model_id=json_load["model_id"])
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
print("Start to convert the AI model." + " \n\tmodel_id: " + json_load["model_id"])

## Get AI model status after conversion

Converting takes time.<br>

To complete the conversion, ensure that the conversion status is **`Conversion completed`**.

After you start the conversion, run the following code cell to check the status :

In [ ]:
convert_status = get_base_model_status(json_load["model_id"])
print(convert_status + " \n\tmodel_id: " + json_load["model_id"])